In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

import json
import os.path

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

In [3]:
class ModelData:
    
    def __init__(self, data_prep_params, name, history ):
        self.data_params = data_prep_params
        self.name = name
        self.history = history

        
    

In [4]:
def generate_options(q = None, normed = None, reco = None, cosz = None):
    
    retVal = {}
    i = 0
    
    for charge in q:
        for norm in normed:
            for rec in reco:
                for cos_zen in cosz:
                        retVal['Model_' + str(i)] = {'q': charge, 't': False, 'normed': norm, 'reco': rec, 'cosz': cos_zen}
                        i = i + 1

    return retVal
        
    

In [5]:
x, y = load_preprocessed(simPrefix, 'train')
#options = {'q':None, 't':False, 'normed':True}

# mergetypes['mean','sum','product','min','max']

Percentage of events with a NaN: 2.68


In [26]:
#set what options to try exhaustively c
q = [None,'mean','sum','product','min','max']
normed = [True, False]
#reco = [ None, 'plane', 'laputop', 'small']
#cosz = [False, True]


folder_path = 'C:/Users/Caden H/Desktop/icecube/github_dir/icetop-cnn/data_baseline_models'

models = []

options = generate_options(q, normed , [None] , [False])


subset = 1000
i = 0
for option in options:
    
    
    name = 'Model_' + str(i)
    prep = options.get(option)


    model = Sequential(name = nameModel(prep, name)) # i dont know what the nameModel object does exactly
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = None))
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
    model.add(Flatten())
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

    energy = y['energy']
    y['dir'] = np.array(y['dir'])
    comp = y['comp']
    theta, phi = y['dir'].transpose()
    nevents = len(energy)
    trainCut = (np.random.uniform(size=nevents) < 0.85)
    testCut = np.logical_not(trainCut)

    # Establish arrays to be trained on
    x_i = dataPrep(x, y, **prep)
    temp_y = energy
    print('Training model ' + name + ':' )
    history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3)
    
    
    models.append(ModelData(prep, name, history))
    
    model.save(os.path.join(folder_path, name +'.h5'))
    print(name + 'saved!')
    
    if i == subset:
        break
    else:
        i = i + 1



Training model Model_0:
Epoch 1/3
14604/14604 [==============================] - 855s 58ms/step - loss: 0.1781 - mse: 0.1781 - val_loss: 0.0585 - val_mse: 0.0585
Epoch 2/3
14604/14604 [==============================] - 46s 3ms/step - loss: 0.0589 - mse: 0.0589 - val_loss: 0.0566 - val_mse: 0.0566
Epoch 3/3
14604/14604 [==============================] - 47s 3ms/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.0512 - val_mse: 0.0512
Model_0saved!
Training model Model_1:
Epoch 1/3
14604/14604 [==============================] - 45s 3ms/step - loss: 0.1565 - mse: 0.1565 - val_loss: 0.0653 - val_mse: 0.0653
Epoch 2/3
14604/14604 [==============================] - 38s 3ms/step - loss: 0.0558 - mse: 0.0558 - val_loss: 0.0483 - val_mse: 0.0483
Epoch 3/3
14604/14604 [==============================] - 28s 2ms/step - loss: 0.0504 - mse: 0.0504 - val_loss: 0.0468 - val_mse: 0.0468
Model_1saved!


C:\Users\Caden H\Desktop\icecube\github_dir\icetop-cnn\data_tools.py:265: RuntimeWarning: Mean of empty slice
  new_out = np.nanmean([a1,a2], axis=0)


Training model Model_2:
Epoch 1/3


ValueError: in user code:

    C:\Users\Caden H\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Caden H\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Caden H\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Caden H\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 

In [8]:
print(options.get('Model_1'))

None


In [18]:
import pickle

    
file = open('Performance_Data.pickle' , 'wb')
pickle.dump(models, file)


TypeError: cannot pickle 'weakref' object

In [24]:
i = 0
for model in models:

    print(str(i) + ':' + str(model.history.history.get('loss')))
    i = i + 1



0:[0.1945875883102417]
1:[0.15266022086143494]


In [29]:
print(options.get('Model_2'))

{'q': 'mean', 't': False, 'normed': True, 'reco': None, 'cosz': False}
